# Introdução

Este trabalho apresenta uma aplicação do modelo BERT na tarefa de classificação de texto.

Será analisando o banco de dados X extraído do site Kagle. Este banco possuí duas colunas, uma com o texto da notícia e outra indicando se a notícia é falsa ou não.

Utilizaremos neste trabalho a linguagem `Python` (versão 3.11.3), `R` (versão 4.2.1), o software `RStudio` (versão 2023.06.0+421), os pacotes em python `pandas` (versão 2.0.3), `transformers` (versão 4.30.2), `torch` (versão 2.0.1), e os pacotes em R .

# Setup

In [1]:
# Importando os pacotes
import os
import time
import datetime
import random
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from statistics import mean
from collections import Counter
from langdetect import detect, DetectorFactory
from sklearn.metrics import matthews_corrcoef
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, get_linear_schedule_with_warmup

c:\Users\Gabriel\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Declarando constantes
os.environ['CURL_CA_BUNDLE'] = ''  # Caso surja "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed" rode este comando e reduza a versão do pacote requets: "pip uninstall requests" e "pip install requests==2.27.1"

CAMINHO_ORIGINAL = r"dados\WELFake_Dataset.csv"
CAMINHO_FINAL = r"dados\data.csv"

In [ ]:
# Declarando semente aleatória para que este código seja reprodutível
my_seed = 288933

random.seed(my_seed)
np.random.seed(my_seed)
torch.manual_seed(my_seed)
torch.cuda.manual_seed_all(my_seed)

# Pré-Processamento

O banco original possui 3 colunas e 72134 observações, das quais 35028 são notícias verdadeiras (label=0) e 37106 são falsas (label=1).

In [38]:
# # Importando o banco
# df = pd.read_csv(CAMINHO_ORIGINAL, index_col=0)

# df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 72134 entries, 0 to 72133
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   71576 non-null  object
 1   text    72095 non-null  object
 2   label   72134 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [34]:
# df.groupby('label').size()

label
0    35028
1    37106
dtype: int64

In [39]:
# # Renomeando a coluna de título e selecionando somente as colunas de título e de classificação
# df.rename({'title': 'titulo'}, axis=1, inplace=True)
# df = df[['titulo', 'label']]

# n_linhas_antes = len(df)
# df = df[~df['titulo'].isnull()]
# df = df[~df['label'].isnull()]

# n_linhas_depois = len(df)
# print(f'Número de observações removidas por valores nulos nas colunas de título ou de classificação: {n_linhas_antes- n_linhas_depois}')

Número de observações removidas por valores nulos nas colunas de título ou de classificação: 558


In [40]:
# # Removendo linhas duplicadas
# n_linhas_antes = len(df)
# df.drop_duplicates(inplace=True)  # Valores de todas as colunas iguais
# df.drop_duplicates(subset='titulo', keep=False, inplace=True, ignore_index=True)  # Valores removidos aqui são títulos que possuem ambos os labels

# n_linhas_depois = len(df)
# print(f'Número de observações removidas por valores duplicados: {n_linhas_antes- n_linhas_depois}')

Número de observações removidas por valores duplicados: 9233


Alguns títulos estão em outras línguas além de inglês. Para removê-los, utilizaremos o pacote `langdetect` (versão 1.0.9).

In [26]:
# # Classificando os títulos em línguas
# DetectorFactory.seed = 0

# lingua = []
# for title in df['titulo']:
#     try:
#         lingua.append(detect(title))
#     except:
#         lingua.append('error')
#         pass

# print(Counter(lingua))

786
8709
17718
35080
39091
40841
47261
Counter({'en': 60048, 'de': 814, 'fr': 212, 'es': 204, 'ru': 153, 'ca': 152, 'it': 116, 'nl': 96, 'af': 73, 'da': 72, 'no': 68, 'id': 64, 'ro': 50, 'pt': 41, 'et': 27, 'sv': 24, 'tl': 21, 'ar': 19, 'vi': 15, 'lt': 11, 'so': 10, 'tr': 10, 'error': 7, 'hr': 7, 'cy': 6, 'fi': 4, 'pl': 3, 'sl': 3, 'sw': 3, 'bg': 3, 'el': 2, 'lv': 1, 'fa': 1, 'sq': 1, 'zh-cn': 1, 'hu': 1})


Vamos excluir as linhas em que o pacote `langdetect` classificou os títulos como língua russa, árabe, lituana, turca, polonesa, búlgara, grega, chinesa ou títulos que não puderam ser classificados pelo pacote. Vários títulos em inglês foram classificados nas demais línguas, e por tanto consideramos que tal classificação não foi muito acurada. Além disso, é importante ter em mente que vários títulos em línguas que não o inglês não puderam ser removidas do banco de forma automática, o que poderá provocar um ruído no modelo ajustado.

In [87]:
# # Removendo títulos em outras línguas
# n_linhas_antes = len(df)
# df.drop([idx for idx, l in enumerate(lingua) if l in ['ru', 'ar', 'lt', 'tr', 'pl', 'bg', 'el', 'zh-cn', 'error']]
#         ,axis=0  # linhas
#         ,inplace=True)
# df.reset_index(drop=True, inplace=True)

# n_linhas_depois = len(df)
# print(f'Número de títulos removidos por não estarem em inglês: {n_linhas_antes- n_linhas_depois}')

Número de títulos removidos por não estarem em inglês: 209


Para analisar o tamanho de cada título, vamos considerar o número de tokens do título tokenizado pelo modelo pré-treinado BERT Base. A informação do número máximo de tokens será útil na fase de ajuste do modelo.

In [12]:
# # Declarando o tokenizador do modelo BERT Base Uncased (que considera todas as letras como minúsculas)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Para cada título, calcule o tamanho do vetor de tokens
# df['n_token'] = [len(tokenizer.encode(title, add_special_tokens=True)) for title in df['titulo']]

# # Analisando algumas estatísticas
# print('Mínimo de tokens: ', min(df['n_token']))
# print('Máximo de tokens: ', max(df['n_token']))
# print('Média de tokens: ', round(mean(df['n_token']), 4))
# print('Quantidade de tokens por percentil: ', np.quantile(df['n_token'], q=[.1, .2, .3, .4, .5, .6, .7, .8, .9]))
# print('Número de títulos com mais de 128 tokens: ', sum([i > 128 for i in df['n_token']]))
# print('Número de títulos com mais de 64 tokens: ', sum([i > 64 for i in df['n_token']]))
# print('Número de títulos com mais de 32 tokens: ', sum([i > 32 for i in df['n_token']]))

Mínimo de tokens:  3
Máximo de tokens:  96
Média de tokens:  18.1779
Quantidade de tokens por percentil:  [12. 13. 15. 16. 17. 19. 20. 22. 25.]
Número de títulos com mais de 128 tokens:  0
Número de títulos com mais de 64 tokens:  3
Número de títulos com mais de 32 tokens:  1415


In [105]:
# # Banco pré-processado
# df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 62134 entries, 0 to 62342
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   titulo   62134 non-null  object
 1   label    62134 non-null  int64 
 2   n_token  62134 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.9+ MB


,titulo,label,n_token
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,35
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,30
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0,22
3,SATAN 2: Russia unvelis an image of its terrif...,1,25
4,About Time! Christian Group Sues Amazon and SP...,1,18


In [6]:
# df.groupby('label').size()

label
0    34404
1    27730
dtype: int64

O banco final possui 62134 observações, sendo que 55% delas são notícias falsas e 45% notícias verdadeiras, aproximadamente. Com o banco pré-processado pronto, vamos exportá-lo para um arquivo CSV.

In [108]:
# df.to_csv(CAMINHO_FINAL, index=False)

# Estatísticas descritivas

Importando os dados processados

In [3]:
df = pd.read_csv(CAMINHO_FINAL)

In [4]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62134 entries, 0 to 62133
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   titulo   62134 non-null  object
 1   label    62134 non-null  int64 
 2   n_token  62134 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


,titulo,label,n_token
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,35
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,30
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0,22
3,SATAN 2: Russia unvelis an image of its terrif...,1,25
4,About Time! Christian Group Sues Amazon and SP...,1,18


Obtendo a distribuição de palavras de cada texto

In [7]:
df.groupby('label').size()

label
0    34404
1    27730
dtype: int64

## Importando o modelo BERT pré-treinado

Vamos importar o modelo BERT base, pois não dispomos de capacidade computacional suficiente para rodar o modelo BERT large. Mais especificamente, utilizaremos o modelo BERT base uncased, que não faz diferença entre palavras com letras maiúsculas e minusculas.

In [13]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
#model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Change num_labels according to your classification task


KeyboardInterrupt



In [ ]:
# Tokenize and preprocess input text
def preprocess_text(text):
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        truncation=True,
        padding='max_length',
        max_length=512,  # Adjust according to your input length requirements
        return_tensors='pt'
    )
    return inputs

# Example text for classification
text = "This is an example sentence for classification."

# Preprocess input text
inputs = tokenizer.encode_plus('text                  a   q',#df['text'][5],
        add_special_tokens=True,
        truncation=True,
        padding='max_length',
        max_length=512)
print(inputs['input_ids'])
print(tokenizer.convert_ids_to_tokens(inputs['input_ids']))
print(len([i for i in inputs['input_ids'] if i != 0]))

NameError: name 'tokenizer' is not defined

In [72]:
df.head()

,text,flag,tamanho
0,the head of a conservative republican faction ...,1,746
1,transgender people will be allowed for the fir...,1,396
2,the special counsel investigation of links bet...,1,454
3,trump campaign adviser george papadopoulos tol...,1,373
4,president donald trump called on the u.s. post...,1,849
